Imported different libraries used to perform Association rules,different types of recommendations(collabrative , content and hybrid based recommendation systems)

In [ ]:

import os
import pandas as pd
import kagglehub
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import fpgrowth, association_rules
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


From kaggle groceries-dataset is exported and kept that dataset in to the dataframe 'df' using pandas library, and performed different operations on the dataset to clean the dataset.

In [ ]:

path = kagglehub.dataset_download("heeraldedhia/groceries-dataset")

print("Path to dataset files:", path)
csv_file=None
for file in os.listdir(path):
    if file.endswith(".csv"):
        csv_file = os.path.join(path, file)
        break
if csv_file is None:
    print("No CSV file found in the downloaded dataset.")
else:
  print(f"Found CSV file: {csv_file}")
df = pd.read_csv(csv_file)
print(df.head())
print(df.info())
print(df.describe())
print(df.columns)
print(df.duplicated())
print(df.shape)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Path to dataset files: /root/.cache/kagglehub/datasets/heeraldedhia/groceries-dataset/versions/1
Found CSV file: /root/.cache/kagglehub/datasets/heeraldedhia/groceries-dataset/versions/1/Groceries_dataset.csv
   Member_number        Date   itemDescription
0           1808  21-07-2015    tropical fruit
1           2552  05-01-2015        whole milk
2           2300  19-09-2015         pip fruit
3           1187  12-12-2015  other vegetables
4           3037  01-02-2015        whole milk
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB
None
       Member_number
count   38765.000000
mean     3003.641868
std      1153.611031
min      1000.000000
25%      2002.

This code is implementing an association rule mining approach using the apriori algorithm to find frequent itemsets and generate association rules. Here's a step-by-step interpretation:


One-Hot Encoding the Data:

The code assumes a dataset (df) with columns 'Member_number' and 'itemDescription', where each row represents an item purchased by a particular member.

df.groupby(['Member_number', 'itemDescription'])['itemDescription'].count().unstack():

This line groups the data by 'Member_number' and 'itemDescription' to create a pivoted, transaction-item matrix.
Each cell shows the count of each item for each member.
.fillna(0).set_index('Member_number'):

Missing values (where an item was not purchased by a member) are filled with zeros, and 'Member_number' is set as the index.
Binary Encoding:

df_encoded = df_encoded.map(lambda x: 1 if x > 0 else 0): Converts each count to binary (1 if the item was purchased by the member and 0 otherwise).
Result:

This results in a binary-encoded dataframe (df_encoded) where each row is a transaction, each column is an item, and values are 1 if the item was purchased in that transaction and 0 otherwise.
Mining Frequent Itemsets:

frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True):

Applies the apriori algorithm on the binary-encoded dataset to identify frequent itemsets (combinations of items frequently purchased together).
min_support=0.01 sets the minimum support threshold to 1%, meaning an itemset must appear in at least 1% of transactions to be considered frequent.
use_colnames=True ensures item names are displayed instead of column indices.
Printing Results:

print("Frequent itemsets:") and print(frequent_itemsets) display the identified frequent itemsets and their support values.
Finding Itemsets with the Highest Support:

high_support_value = frequent_itemsets['support'].max(): Determines the highest support value among all frequent itemsets.
highest_support_itemsets = frequent_itemsets[frequent_itemsets['support'] == high_support_value]: Filters itemsets with the highest support value.
Printing:
print("Itemsets with the Highest Support:") and print(highest_support_itemsets) display itemsets that have the maximum support, indicating items that most frequently co-occur in transactions.
Generating Association Rules:

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5):
This line uses the frequent itemsets to generate association rules.
metric="confidence" specifies that rules should be evaluated based on confidence (likelihood of buying consequent if antecedent is purchased).
min_threshold=0.5 sets a confidence threshold of 50%, meaning only rules with a confidence of 50% or higher are kept.
Displaying and Interpreting the Association Rules:

print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]):
This prints the association rules, showing:
antecedents: The item(s) on the left-hand side of the rule (if a member buys these).
consequents: The item(s) on the right-hand side of the rule (they are likely to buy these as well).
support: The proportion of transactions containing both antecedent and consequent items.
confidence: The likelihood of the consequent item being bought when the antecedent item is bought.
lift: Indicates how much more likely the consequent is to be purchased with the antecedent compared to random chance. A lift > 1 implies a positive association.

In [ ]:


# Perform one-hot encoding if necessary
# Assuming 'Item' and 'Transaction' columns exist
# Group by 'Transaction' to get the transaction-item format

df_encoded = df.groupby(['Member_number', 'itemDescription'])['itemDescription'].count().unstack().reset_index().fillna(0).set_index('Member_number')
df_encoded = df_encoded.map(lambda x: 1 if x > 0 else 0)

print(df_encoded.head())

# Minimum support threshold for frequent itemsets
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)

print("Frequent itemsets:")
print(frequent_itemsets)

high_support_value=frequent_itemsets['support'].max()
print("High support value:", high_support_value)
highest_support_itemsets = frequent_itemsets[frequent_itemsets['support'] == high_support_value]
print("Itemsets with the Highest Support:")
print(highest_support_itemsets)

# Generate association rules with confidence threshold
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# Display rules sorted by confidence or lift
print("Association rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


itemDescription  Instant food products  UHT-milk  abrasive cleaner  \
Member_number                                                        
1000                                 0         0                 0   
1001                                 0         0                 0   
1002                                 0         0                 0   
1003                                 0         0                 0   
1004                                 0         0                 0   

itemDescription  artif. sweetener  baby cosmetics  bags  baking powder  \
Member_number                                                            
1000                            0               0     0              0   
1001                            0               0     0              0   
1002                            0               0     0              0   
1003                            0               0     0              0   
1004                            0               0     0          

/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Frequent itemsets:
       support                                           itemsets
0     0.015393                            (Instant food products)
1     0.078502                                         (UHT-milk)
2     0.031042                                    (baking powder)
3     0.119548                                             (beef)
4     0.079785                                          (berries)
...        ...                                                ...
3011  0.011031     (soda, whole milk, whipped/sour cream, yogurt)
3012  0.010518  (whole milk, rolls/buns, bottled water, other ...
3013  0.013597  (whole milk, rolls/buns, sausage, other vegeta...
3014  0.010005  (whole milk, rolls/buns, shopping bags, yogurt...
3015  0.013597  (whole milk, rolls/buns, soda, other vegetable...

[3016 rows x 2 columns]
High support value: 0.4581836839404823
Itemsets with the Highest Support:
      support      itemsets
113  0.458184  (whole milk)
Association rules:
               

Here we are sorting the Association rules by confidence in desending order .So that we can say how much percentage the customer will buy the product based on the antecedents

In [ ]:
# Sort rules by confidence in descending order
rules_sorted_by_confidence = rules.sort_values(by='confidence', ascending=False)

print("Association rules sorted by confidence:")
print(rules_sorted_by_confidence[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


Association rules sorted by confidence:
                                            antecedents         consequents  \
410                               (meat, domestic eggs)        (whole milk)   
315                  (fruit/vegetable juice, chocolate)        (whole milk)   
1107  (yogurt, rolls/buns, other vegetables, bottled...        (whole milk)   
795                  (yogurt, bottled water, pip fruit)        (whole milk)   
822                   (rolls/buns, yogurt, brown bread)        (whole milk)   
...                                                 ...                 ...   
477              (fruit/vegetable juice, shopping bags)              (soda)   
510                      (rolls/buns, hygiene articles)        (whole milk)   
246                             (butter, shopping bags)  (other vegetables)   
517                    (soda, long life bakery product)        (whole milk)   
888                      (whole milk, rolls/buns, curd)  (other vegetables)   

       supp

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


This code explain associate rule mining using the FP-Growth algorithm. Which is same as the Apriori Algorithm.

In [ ]:
# Import necessary libraries
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load the dataset (assuming df is already preprocessed and in the required format)
# This step assumes you already have a binary-encoded DataFrame `df_encoded`
# where each transaction is a row and each item is a column with 0/1 indicating absence/presence.

# Step 1: Generate frequent itemsets using FP-Growth
frequent_itemsets = fpgrowth(df_encoded, min_support=0.01, use_colnames=True)

# Display the frequent itemsets
print("Frequent itemsets generated by FP-Growth:")
print(frequent_itemsets)

# Step 2: Generate association rules from the frequent itemsets
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# Step 3: Display the association rules
print("Association rules generated from FP-Growth:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

# Optional: Sort by a specific metric (e.g., confidence or lift) for easier interpretation
rules_sorted = rules.sort_values(by='confidence', ascending=False)
print("Rules sorted by confidence:")
print(rules_sorted[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Frequent itemsets generated by FP-Growth:
       support                               itemsets
0     0.458184                           (whole milk)
1     0.313494                                 (soda)
2     0.282966                               (yogurt)
3     0.206003                              (sausage)
4     0.177527                               (pastry)
...        ...                                    ...
3011  0.013340        (whole milk, seasonal products)
3012  0.010262        (rolls/buns, seasonal products)
3013  0.010005  (other vegetables, seasonal products)
3014  0.010005                 (frozen dessert, soda)
3015  0.012057           (frozen dessert, whole milk)

[3016 rows x 2 columns]
Association rules generated from FP-Growth:
                               antecedents   consequents   support  \
0                       (soda, rolls/buns)  (whole milk)  0.065162   
1     (soda, rolls/buns, other vegetables)  (whole milk)  0.031811   
2                 (soda, other 

This code performs a recommendation process using collaborative filtering. It identifies similar members based on their purchase history and recommends items that these similar members have bought but the target member hasn't.

Key Steps:
Identify Unique Members and Purchases:

Counts unique members and groups their purchased items into lists.
Create a Binary Item-User Matrix:

Expands each member’s purchases into a matrix where each row represents a member and each column represents an item (1 if purchased, 0 otherwise).
Compute Cosine Similarity:

Measures similarity between members based on their item-user matrix, helping identify members with similar buying patterns.
Generate Recommendations:

For a target member, finds the top similar members, collects items they've bought, and recommends items that the target member hasn’t yet purchased.
This approach provides personalized recommendations by leveraging shared buying behavior across similar members.

In [ ]:
# Get all unique 'Member_number's
unique_members = df['Member_number'].unique()
print("All unique 'Member_number's:")
print(unique_members)

# Count the number of unique 'Member_number's
num_unique_members = len(unique_members)
print(f"Number of unique 'Member_number's: {num_unique_members}")

member_transactions = df.groupby('Member_number')['itemDescription'].apply(list).reset_index()

# Rename the column for clarity
member_transactions.columns = ['Member_number', 'Purchased_Items']

# Display the grouped data
print("Items purchased by each member:")
print(member_transactions.head())

# Checking for clusters by each unique transaction
# (This would give insight into items commonly bought together by each user)
print("Total unique member transactions:", member_transactions.shape[0])

# Step 1: Create an item-user matrix
# Explode the Purchased_Items into separate rows for counting
df_exploded = member_transactions.explode('Purchased_Items')

# Create a binary item-user matrix (1 if purchased, 0 otherwise)
item_user_matrix = df_exploded.groupby(['Member_number', 'Purchased_Items']).size().unstack(fill_value=0)
#print("Item-user matrix:")
#print(item_user_matrix.head())


# Compute cosine similarity between members based on their purchased items
cosine_sim = cosine_similarity(item_user_matrix)
cosine_sim_df = pd.DataFrame(cosine_sim, index=item_user_matrix.index, columns=item_user_matrix.index)
print("Cosine similarity matrix:")
print(cosine_sim_df.head())

# Display the cosine similarity matrix
print("Cosine similarity between members:")
print(cosine_sim_df.shape)

# Function to recommend items to a member based on similar members' purchases
def recommend_items(member_id, top_n=5):
    # Get similarity scores for the target member
    similar_members = cosine_sim_df[member_id].sort_values(ascending=False).drop(member_id).head(top_n).index

    # Get items purchased by similar members
    similar_members_items = item_user_matrix.loc[similar_members].sum(axis=0)
    similar_members_items = similar_members_items[similar_members_items > 0].index

    # Get items already purchased by the target member
    target_member_items = set(item_user_matrix.columns[item_user_matrix.loc[member_id] > 0])

    # Recommend items that similar members have purchased but the target member has not
    recommendations = [item for item in similar_members_items if item not in target_member_items]

    print(f"Recommendations for member {member_id}: {recommendations}")
    return recommendations

# Example usage: recommend items for a specific member
member_id = item_user_matrix.index[334]  # Replace with the actual member ID you're interested in
recommend_items(member_id)



All unique 'Member_number's:
[1808 2552 2300 ... 3607 4587 2417]
Number of unique 'Member_number's: 3898
Items purchased by each member:
   Member_number                                    Purchased_Items
0           1000  [soda, canned beer, sausage, sausage, whole mi...
1           1001  [frankfurter, frankfurter, beef, sausage, whol...
2           1002  [tropical fruit, butter milk, butter, frozen v...
3           1003  [sausage, root vegetables, rolls/buns, deterge...
4           1004  [other vegetables, pip fruit, root vegetables,...
Total unique member transactions: 3898


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cosine similarity matrix:
Member_number      1000      1001      1002      1003      1004      1005  \
Member_number                                                               
1000           1.000000  0.457330  0.171499  0.129641  0.379980  0.000000   
1001           0.457330  1.000000  0.166667  0.251976  0.328244  0.288675   
1002           0.171499  0.166667  1.000000  0.000000  0.369274  0.000000   
1003           0.129641  0.251976  0.000000  1.000000  0.325669  0.654654   
1004           0.379980  0.328244  0.369274  0.325669  1.000000  0.284268   

Member_number      1006      1008      1009      1010  ...      4991  \
Member_number                                          ...             
1000           0.211702  0.194461  0.219382  0.000000  ...  0.259281   
1001           0.514344  0.125988  0.000000  0.176777  ...  0.125988   
1002           0.154303  0.094491  0.106600  0.000000  ...  0.283473   
1003           0.349927  0.071429  0.000000  0.200446  ...  0.142857   
10

['UHT-milk',
 'butter',
 'finished products',
 'long life bakery product',
 'processed cheese',
 'sausage',
 'tropical fruit',
 'turkey',
 'yogurt']

this code is to get the all the unique items from the 'itemDescription' column and printing them.

In [ ]:
unique_items = df['itemDescription'].unique()
print("All unique items:")
print(unique_items)

# Count the number of unique items
num_unique_items = len(unique_items)
print(f"Number of unique items: {num_unique_items}")

All unique items:
['tropical fruit' 'whole milk' 'pip fruit' 'other vegetables' 'rolls/buns'
 'pot plants' 'citrus fruit' 'beef' 'frankfurter' 'chicken' 'butter'
 'fruit/vegetable juice' 'packaged fruit/vegetables' 'chocolate'
 'specialty bar' 'butter milk' 'bottled water' 'yogurt' 'sausage'
 'brown bread' 'hamburger meat' 'root vegetables' 'pork' 'pastry'
 'canned beer' 'berries' 'coffee' 'misc. beverages' 'ham' 'turkey'
 'curd cheese' 'red/blush wine' 'frozen potato products' 'flour' 'sugar'
 'frozen meals' 'herbs' 'soda' 'detergent' 'grapes' 'processed cheese'
 'fish' 'sparkling wine' 'newspapers' 'curd' 'pasta' 'popcorn'
 'finished products' 'beverages' 'bottled beer' 'dessert' 'dog food'
 'specialty chocolate' 'condensed milk' 'cleaner' 'white wine' 'meat'
 'ice cream' 'hard cheese' 'cream cheese ' 'liquor' 'pickled vegetables'
 'liquor (appetizer)' 'UHT-milk' 'candy' 'onions' 'hair spray'
 'photo/film' 'domestic eggs' 'margarine' 'shopping bags' 'salt' 'oil'
 'whipped/sour cream'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


This code implements a category-based recommendation system that uses TF-IDF (Term Frequency-Inverse Document Frequency) and cosine similarity to recommend items to members based on what similar members have purchased. Here's a summary of each step:

Steps Explained:
Map Items to Categories:

Each item in the dataset is mapped to a predefined category (e.g., "Fruits and Vegetables," "Dairy") using a categories dictionary.
A new column, Category, is added to the DataFrame to store the mapped category for each item.
Group Categories by Member:

Items purchased by each member are grouped by member ID, creating a list of categories for each member's purchases.
Duplicate categories within each member's list are removed for uniqueness, resulting in one occurrence of each category in each list.
Prepare Data for TF-IDF Vectorization:

A new column, Category_Text, is created by joining each member's unique categories into a single string.
This text format allows TF-IDF vectorization of each member's purchasing patterns in terms of categories.
Compute Cosine Similarity Between Members:

The TF-IDF vectorizer converts the Category_Text data into numerical vectors, capturing the importance of each category for each member.
Cosine similarity is computed between these vectors to measure the similarity in purchasing patterns across members.
The result is a similarity matrix, with each entry indicating the similarity score between two members.
Generate Item Recommendations:

For a target member, the function recommend_items() finds the most similar members based on their category similarity.
It gathers items purchased by similar members, removes items the target member has already bought, and recommends new items to the target member.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Define the categories dictionary
categories = {
    'Fruits and Vegetables': [
        'tropical fruit', 'pip fruit', 'citrus fruit', 'root vegetables', 'berries', 'grapes', 'onions', 'specialty vegetables'
    ],
    'Dairy': [
        'whole milk', 'butter', 'yogurt', 'curd cheese', 'cream cheese', 'hard cheese', 'processed cheese',
        'sliced cheese', 'spread cheese', 'whipped/sour cream', 'UHT-milk'
    ],
    'Bakery': [
        'rolls/buns', 'brown bread', 'pastry', 'white bread', 'semi-finished bread', 'long life bakery product'
    ],
    'Meat and Poultry': [
        'beef', 'chicken', 'hamburger meat', 'pork', 'sausage', 'turkey', 'meat', 'organic sausage'
    ],
    'Fish and Seafood': [
        'fish', 'frozen fish', 'canned fish'
    ],
    'Frozen Foods': [
        'frozen potato products', 'frozen meals', 'frozen vegetables', 'frozen chicken', 'frozen dessert', 'frozen fruits'
    ],
    'Beverages': [
        'fruit/vegetable juice', 'bottled water', 'soda', 'misc. beverages', 'beverages', 'canned beer',
        'bottled beer', 'red/blush wine', 'white wine', 'sparkling wine', 'liquor', 'prosecco', 'rum',
        'brandy', 'liqueur', 'whisky'
    ],
    'Snacks and Confectionery': [
        'chocolate', 'specialty bar', 'candy', 'salty snack', 'chewing gum', 'popcorn', 'snack products',
        'chocolate marshmallow', 'cake bar', 'sweet spreads', 'nut snack'
    ],
    'Canned and Jarred Goods': [
        'canned vegetables', 'canned fish', 'canned fruit', 'condensed milk', 'pickled vegetables',
        'preservation products'
    ],
    'Condiments and Sauces': [
        'mustard', 'ketchup', 'salad dressing', 'sauces', 'mayonnaise'
    ],
    'Pantry Staples': [
        'flour', 'sugar', 'baking powder', 'rice', 'salt', 'oil', 'vinegar', 'herbs', 'spices', 'honey',
        'jam', 'instant coffee', 'tea', 'cocoa drinks', 'cereals'
    ],
    'Prepared and Convenience Foods': [
        'instant food products', 'frozen meals', 'soups', 'ready soups', 'pudding powder'
    ],
    'Personal Care and Hygiene': [
        'toilet cleaner', 'dental care', 'soap', 'male cosmetics', 'female sanitary products', 'skin care',
        'make up remover', 'hygiene articles', 'baby cosmetics', 'hair spray'
    ],
    'Household Items': [
        'cleaner', 'detergent', 'dish cleaner', 'house keeping products', 'softener', 'kitchen towels',
        'cling film/bags', 'kitchen utensil', 'abrasive cleaner', 'rubbing alcohol', 'light bulbs'
    ],
    'Pet Care': [
        'dog food', 'cat food', 'pet care'
    ],
    'Miscellaneous': [
        'newspapers', 'shopping bags', 'candles', 'flower soil/fertilizer', 'flower (seeds)', 'photo/film',
        'seasonal products', 'dishes', 'cooking chocolate'
    ]
}

# Define a function to map each item to its category
def map_to_category(item):
    for category, items in categories.items():
        if item in items:
            return category
    return 'Unknown'  # If item does not match any category

# Apply the function to the 'itemDescription' column
df['Category'] = df['itemDescription'].apply(map_to_category)

# Display the updated DataFrame with the Category column
print(df.head())
print(df['Category'])
print(df.info())
item_category = df.groupby('Member_number')['Category'].apply(list).reset_index()

# Rename the column for clarity
item_category.columns = ['Member_number', 'Categories']

# Display the grouped data
print("Items purchased by each member:")
print(item_category.head())

# Remove duplicate items in each category list by converting each list to a set and then back to a list


# Remove duplicate categories in each list by converting each list to a set and then back to a list
item_category['Categories'] = item_category['Categories'].apply(lambda x: list(set(x)))

# Check the updated categories dictionary
print(item_category.head())
# Step 2: Prepare category text data for TF-IDF
item_category['Category_Text'] = item_category['Categories'].apply(lambda x: ' '.join(x))

# Display the updated DataFrame
print(item_category.head())
# Step 3: Vectorize category data
vectorizer = TfidfVectorizer()
category_matrix = vectorizer.fit_transform(item_category['Category_Text'])
# Step 4: Compute cosine similarity matrix
similarity_matrix = cosine_similarity(category_matrix)

# Convert to DataFrame for readability
similarity_df = pd.DataFrame(similarity_matrix, index=item_category['Member_number'], columns=item_category['Member_number'])
print("Similarity matrix between members based on item categories:")
print(similarity_df)
member_transactions['Purchased_Items'] = member_transactions['Purchased_Items'].apply(lambda x: list(set(x)))
print(member_transactions.head())

# Step 5: Recommend items based on similar members' purchases
def recommend_items(member_id, member_transactions, similarity_matrix, top_n=3):
    # Get the index of the given member
    # CHANGED: Corrected DataFrame name to item_category
    member_idx = member_transactions[member_transactions['Member_number'] == member_id].index[0]

    # Get similarity scores for the member with all other members
    similarity_scores = similarity_matrix[member_idx]

    # Find the most similar members, excluding the member itself
    similar_member_indices = similarity_scores.argsort()[-top_n-1:-1][::-1]

    # Gather items from similar members
    recommended_items = set()
    for idx in similar_member_indices:
        recommended_items.update(member_transactions.iloc[idx]['Purchased_Items'])

    # Remove items the member has already purchased
    # CHANGED: Corrected DataFrame name to item_category
    purchased_items = set(member_transactions.iloc[member_idx]['Purchased_Items'])
    recommended_items.difference_update(purchased_items)

    return list(recommended_items)[:top_n]

# Example: Get recommendations for member 1
member_id = 1000
# CHANGED: Corrected DataFrame name to item_category
recommendations = recommend_items(member_id, member_transactions, similarity_matrix)
print(f"Recommendations for member {member_id}: {recommendations}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   Member_number        Date   itemDescription               Category
0           1808  21-07-2015    tropical fruit  Fruits and Vegetables
1           2552  05-01-2015        whole milk                  Dairy
2           2300  19-09-2015         pip fruit  Fruits and Vegetables
3           1187  12-12-2015  other vegetables                Unknown
4           3037  01-02-2015        whole milk                  Dairy
0           Fruits and Vegetables
1                           Dairy
2           Fruits and Vegetables
3                         Unknown
4                           Dairy
                   ...           
38760                       Dairy
38761    Snacks and Confectionery
38762    Snacks and Confectionery
38763                   Beverages
38764                    Pet Care
Name: Category, Length: 38765, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ---